In [1]:
import os
import re
import bs4
import time
import requests
import pprint

In [2]:
def load(url):
    res = requests.get(url)
    res.raise_for_status()

    return res.text

def pickup_tag(html, find_tag):
    soup = bs4.BeautifulSoup(str(html), 'html.parser')
    paragraphs = soup.find_all(find_tag)

    return paragraphs

def parse(html):
    soup = bs4.BeautifulSoup(str(html), 'html.parser')
    # htmlタグの排除
    kashi_row = soup.getText()
    kashi_row = kashi_row.replace('\n', '')
    kashi_row = kashi_row.replace('　', '')

    # 英数字の排除
    kashi_row = re.sub(r'[a-zA-Z0-9]', '', kashi_row)
    # 記号の排除
    kashi_row = re.sub(r'[ ＜＞♪`‘’“”・…_！？!-/:-@[-`{-~]', '', kashi_row)
    # 注意書きの排除
    kashi = re.sub(r'注意：.+', '', kashi_row)

    return kashi

In [3]:
def main():
    with open('kashi.txt', 'a') as f:
        # アーティストページのアドレス
#         url = f'https://w.atwiki.jp/touhoukashi/tag/%E7%A0%95%E6%9C%88'
        url =f'https://'
        # 曲ページの先頭アドレス
        base_url = f'https://www.uta-net.com'
        base_url = f'https://w.atwiki.jp/touhoukashi/'
        base_url = f'https:'
#         base_url = f'https://w.atwiki.jp/touhoukashi/editx/'

        # ページの取得
        html = load(url)

        # 曲ごとのurlを格納
        musics_url = []
        s_url = []
        # 歌詞を格納
        kashis = ''

        """ 曲のurlを取得 """
        # td要素の取り出し
#         for td in pickup_tag(html, 'div'):
        for td in pickup_tag(html, 'li'):
            # a要素の取り出し
            for a in pickup_tag(td, 'a'):
                # href属性にsongを含むか
                if 'pages' in a.get('href'): 
                    if 'guide' in a.get('href'):
                        continue
                    if 'login' in a.get('href'):
                        continue
                    if a.get('title'):
                        continue
                    # urlを配列に追加
                    list = []
                    tmp = ''
                    s_url = a.get('href')
                    for x in s_url:
                        if(x != '/'):
                            tmp += x
                        else:
                            list.append(tmp)
                            tmp = ''
                    list.append(tmp)
#                     print(list)
#                     print(list[-1])
#                     s_url = a.get('href')
#                     print(s_url)
#                     s_url.split('/')
#                     print(s_url[3])
#                     musics_url.append(base_url + a.get('href'))
                    musics_url.append(base_url + list[-1])
#                     print(musics_url)
                    break
        # pprint.pprint(musics_url)

        """ 歌詞の取得 """
#         soup = bs4.BeautifulSoup(str(html), 'html.parser')
#         ginga =  soup.findAll('div' , class_= 'lyrics')
        for i, page in enumerate(musics_url):
        #この下はテスト用
#             if i == 1:
#                 break
            print('{}曲目:{}'.format(i + 1, page))
            html = load(page)
            for div in pickup_tag(html, 'textarea'):
                # id検索がうまく行えなかった為、一度strにキャスト
                div = str(div)
                # 歌詞が格納されているdiv要素か
#                 if r'itemprop="text"' in div:
#                 if 'lyrics' in div:

#                 latStart = div.find('#divid(lyrics){') + 1
                latStart = div.find('#divid(lyrics){') + 17
                    # つぎに取得の終わりの位置を取得します。ここでポイントは２つ目の引数に入れている、
                    # 検索の開始位置です。'@'以降で一つ目の','の位置を教えてくれます。
#                 latFinish = div.find('</textarea>',latStart)
                latFinish = div.find('}',latStart)
                
                    # 位置を指定(何文字目から何文字目まで)して取得します
                latitude = div[latStart:latFinish]
                    
                    # 不要なデータを取り除く
#                 kashi = parse(div)
#                 kashi = parse(latitude)
#                 print(kashi, end = '\n\n')
                print(latitude, end = '\n\n')
                    # 歌詞を１つにまとめる
#                 kashis += kashi + '\n'
                kashis += latitude + '\n'

                    # １秒待機
                time.sleep(1)
                break
        # 歌詞の書き込み
        f.write(kashis)

if __name__ == '__main__':
    main()

NameError: name 'url' is not defined